In [43]:
import influxdb
import pandas as pd
from influxdb import InfluxDBClient
import json
from datetime import datetime

In [44]:
telegraf_auth = json.load(open('telegraf_credentials'))
telegraf_client = InfluxDBClient(
    host="qtinfluxdb01.intra.qt.io",
    username=telegraf_auth['username'],
    password=telegraf_auth['password'],
    port=8086,
    ssl=True,
    database="telegraf")

<h2>Open nebula waiting times resampled </h2>
This notebooks retireves data about waiting times from opennebula, cleans it (removes nan and values <=0), 
resamples is by 10 minutes and creates 2 files:  
mean values (grouped by 10 min)  
max values (grouped by 10 min)

In [45]:
q = 'SELECT * from "onedb" WHERE  time > now() - 12d'
rs = telegraf_client.query(q)
tdf = pd.DataFrame(rs.get_points())


/home/anwojcie/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [46]:
tdf.shape

(194113, 14)

In [48]:
df = tdf.loc[(tdf['instance'] == "one-master-202105")]
df = df[['time','wtime']]
df = df.dropna()
df = df.loc[(df['wtime'] > 0)]
df.wtime = df.wtime.apply(lambda d: int(d))
df.time = pd.to_datetime(df.time)
df.shape

(193776, 2)

In [49]:
test = df.set_index('time')

In [56]:
test.head()

,wtime
time,
2022-01-14 16:01:40+00:00,772
2022-01-14 16:01:42+00:00,770
2022-01-14 16:01:42+00:00,770
2022-01-14 16:01:42+00:00,771
2022-01-14 16:01:42+00:00,771


In [62]:
df_mean= test.resample('10min').mean()
df_mean = df_mean.dropna()
df_mean.wtime = df_mean.wtime.apply(lambda d: int(d))
df_mean.shape


(1621, 1)

In [63]:
df_mean.head()

,wtime
time,
2022-01-14 16:00:00+00:00,821
2022-01-14 16:10:00+00:00,484
2022-01-14 16:20:00+00:00,442
2022-01-14 16:30:00+00:00,566
2022-01-14 16:40:00+00:00,685


In [64]:
df_mean.to_csv('mean_12_days.csv', index=True)

In [73]:
df_max= test.resample('10min').max()
df_max = df_max.dropna()
df_max.wtime = df_max.wtime.apply(lambda d: int(d))
df_max.shape

(1621, 1)

In [74]:
df_max.head()

,wtime
time,
2022-01-14 16:00:00+00:00,2522
2022-01-14 16:10:00+00:00,2495
2022-01-14 16:20:00+00:00,4679
2022-01-14 16:30:00+00:00,2721
2022-01-14 16:40:00+00:00,2853


In [75]:
df_max.to_csv('max_12_days.csv', index=True)


In [39]:
test_std = test.resample('10min').std()

Just curious how big is standar deviation in this data...

In [41]:
test_mean.head()

,wtime
time,
2022-01-19 15:40:00+00:00,5365.133333
2022-01-19 15:50:00+00:00,4742.971429
2022-01-19 16:00:00+00:00,5385.833962
2022-01-19 16:10:00+00:00,5423.511327
2022-01-19 16:20:00+00:00,4218.017730


In [42]:
test_std.head()

,wtime
time,
2022-01-19 15:40:00+00:00,55.200856
2022-01-19 15:50:00+00:00,1904.939667
2022-01-19 16:00:00+00:00,1473.208402
2022-01-19 16:10:00+00:00,1595.815969
2022-01-19 16:20:00+00:00,2712.988567
